In [1]:
import os
import struct
import torch
import numpy as np
from tqdm import tqdm

def read_aedat_file(filename):
    """Read a single .aedat file and return parsed data"""
    with open(filename, 'rb') as f:
        header_lines = []
        while True:
            pos = f.tell()
            line = f.readline()
            if not line:
                raise ValueError(f"No binary data section found in file {filename}, please check file format")

            try:
                decoded_line = line.decode('ascii', errors='strict')
            except UnicodeDecodeError:
                # Unable to decode ASCII, means this is the start of binary data
                f.seek(pos)
                break

            stripped_line = decoded_line.strip()
            if stripped_line.startswith('#'):
                header_lines.append(stripped_line)
            else:
                f.seek(pos)
                break

        data_start_index = f.tell()  # Data section start offset
        data = f.read()

    event_size = 8
    num_events = len(data) // event_size

    addresses = []
    timestamps = []
    xs = []
    ys = []
    polarities = []

    for i in range(num_events):
        event_data = data[i * event_size:(i + 1) * event_size]
        # Parse address and timestamp in big-endian order
        address, t = struct.unpack('>ii', event_data)
        polarity = address & 1
        x = (address >> 1) & 0x7F
        y = (address >> 8) & 0x7F

        addresses.append(address)
        timestamps.append(t)
        xs.append(x)
        ys.append(y)
        polarities.append(polarity)

    return {
        'header': header_lines,
        'addresses': addresses,
        'timestamps': timestamps,
        'xs': xs,
        'ys': ys,
        'polarities': polarities
    }


def process_and_save_voxel(input_base_folder, output_base_folder, grid_size=(64, 64, 3)):
    """
    Process all .aedat files in category folders, convert event data to voxel representation and save as PyTorch tensors.

    Parameters:
        input_base_folder (str): Input folder path containing .aedat files organized by category.
        output_base_folder (str): Output folder path where processed voxel tensors will be saved.
        grid_size (tuple): Resolution of voxel grid, default (32, 32, 32).
    """
    if not os.path.exists(output_base_folder):
        os.makedirs(output_base_folder)

    # Iterate through category folders
    for class_folder in os.listdir(input_base_folder):
        input_folder = os.path.join(input_base_folder, class_folder)
        output_folder = os.path.join(output_base_folder, class_folder)

        if not os.path.isdir(input_folder):
            continue

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for filename in tqdm(os.listdir(input_folder), desc=f"Processing {class_folder}"):
            if not filename.endswith('.aedat'):
                continue

            filepath = os.path.join(input_folder, filename)
            output_filepath = os.path.join(output_folder, filename.replace('.aedat', '.pt'))

            # Read .aedat file
            data = read_aedat_file(filepath)
            xs, ys, timestamps = data['xs'], data['ys'], data['timestamps']

            # Get range for each axis
            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)
            t_min, t_max = min(timestamps), max(timestamps)

            # Initialize voxel grid
            voxel_grid = np.zeros(grid_size, dtype=int)

            # Map coordinates to voxel grid indices
            x_indices = ((np.array(xs) - x_min) / (x_max - x_min) * (grid_size[0] - 1)).astype(int)
            y_indices = ((np.array(ys) - y_min) / (y_max - y_min) * (grid_size[1] - 1)).astype(int)
            t_indices = ((np.array(timestamps) - t_min) / (t_max - t_min) * (grid_size[2] - 1)).astype(int)

            # Count events in each voxel
            for x, y, t in zip(x_indices, y_indices, t_indices):
                voxel_grid[x, y, t] += 1

            # Convert to PyTorch tensor and save
            voxel_tensor = torch.tensor(voxel_grid, dtype=torch.float32)
            torch.save(voxel_tensor, output_filepath)

        print(f"Category '{class_folder}' processed and saved to {output_folder}")

# Example usage
input_base_folder = r"C:\\Users\\Lem17\\Master Thesis\\Data processing\\data_aedat2"
output_base_folder = "./voxel_data_64643"
process_and_save_voxel(input_base_folder, output_base_folder)

Processing .ipynb_checkpoints: 0it [00:00, ?it/s]


Category '.ipynb_checkpoints' processed and saved to ./voxel_data_64643\.ipynb_checkpoints


Processing airplane: 100%|█████████████████████████████████████████████████████████| 1000/1000 [04:00<00:00,  4.17it/s]


Category 'airplane' processed and saved to ./voxel_data_64643\airplane


Processing automobile: 100%|███████████████████████████████████████████████████████| 1000/1000 [05:35<00:00,  2.98it/s]


...

Category 'horse' processed and saved to ./voxel_data_64643\horse


Processing ship: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [04:01<00:00,  4.15it/s]


Category 'ship' processed and saved to ./voxel_data_64643\ship


Processing truck: 100%|████████████████████████████████████████████████████████████| 1000/1000 [05:15<00:00,  3.17it/s]

Category 'truck' processed and saved to ./voxel_data_64643\truck
